In [1]:
import threading
import time

## thread

In [12]:
print(threading.active_count())

6


In [8]:
print(threading.enumerate())

[<_MainThread(MainThread, started 29828)>, <Thread(IOPub, started daemon 28244)>, <Heartbeat(Heartbeat, started daemon 15312)>, <ControlThread(Control, started daemon 20188)>, <HistorySavingThread(IPythonHistorySavingThread, started 35316)>, <ParentPollerWindows(Thread-1, started daemon 6876)>]


In [9]:
print(threading.current_thread())

<_MainThread(MainThread, started 29828)>


In [13]:
def thread_job():
    print(f'This is a added job and the number is {threading.current_thread()}')

In [19]:
added_thread = threading.Thread(target=thread_job)

In [20]:
added_thread.start()

This is a added job and the number is <Thread(Thread-5, started 15588)>


In [21]:
added_thread.start()

RuntimeError: threads can only be started once

In [28]:
def thread_job(name):
    print(f'{name} start')
    for i in range(10):
        time.sleep(0.1)
    print(f'{name} Finish')

In [30]:
def thread_job_no_sleep(name):
    print(f'{name} start')
    print(f'{name} Finish')

In [29]:
added_thread = threading.Thread(target=thread_job,args=("T1",), name='T1')
added_thread.start()
print('All Done')

T1 start
All Done
T1 Finish


## Join

In [25]:
added_thread = threading.Thread(target=thread_job,args=("T1",), name='T1')
added_thread.start()
added_thread.join()
print('All Done')

T1 start
T1 Finish
All Done


In [31]:
thread1 = threading.Thread(target=thread_job,args=("T1",), name='T1')
thread1.start()
thread2 = threading.Thread(target=thread_job_no_sleep,args=("T2",), name='T2')
thread2.start()
print('All Done')

T1 start
T2 start
T2 Finish
All Done
T1 Finish


In [32]:
thread1 = threading.Thread(target=thread_job,args=("T1",), name='T1')
thread1.start()
thread2 = threading.Thread(target=thread_job_no_sleep,args=("T2",), name='T2')
thread2.start()
thread1.join()
print('All Done')

T1 start
T2 start
T2 Finish
T1 Finish
All Done


In [33]:
thread1 = threading.Thread(target=thread_job,args=("T1",), name='T1')
thread1.start()
thread2 = threading.Thread(target=thread_job_no_sleep,args=("T2",), name='T2')
thread2.start()
thread2.join()
print('All Done')

T1 start
T2 start
T2 Finish
All Done
T1 Finish


In [34]:
thread1 = threading.Thread(target=thread_job,args=("T1",), name='T1')
thread1.start()
thread2 = threading.Thread(target=thread_job_no_sleep,args=("T2",), name='T2')
thread2.start()
thread1.join()
thread2.join()
print('All Done')

T1 start
T2 start
T2 Finish
T1 Finish
All Done


## Queue

In [35]:
from queue import Queue

In [36]:
def job(l, q):
    for i in range(len(l)):
        l[i] = l[i] * 2
    q.put(l)

In [41]:
q = Queue()
threads = []
data = [[1,1,1], [2,2,2], [3,3,3], [4,4,4]]
for i in range(4):
    thread = threading.Thread(target=job,args=(data[i],q,), name='T1')
    thread.start()
    threads.append(thread)
for i in range(4):
    threads[i].join()
results = []
for _ in range(4):
    results.append(q.get())
print(results)

[[2, 2, 2], [4, 4, 4], [6, 6, 6], [8, 8, 8]]


## GIL

In [42]:
import threading
from queue import Queue
import copy
import time

def job(l, q):
    res = sum(l)
    q.put(res)

def multithreading(l):
    q = Queue()
    threads = []
    for i in range(4):
        t = threading.Thread(target=job, args=(copy.copy(l), q), name='T%i' % i)
        t.start()
        threads.append(t)
    [t.join() for t in threads]
    total = 0
    for _ in range(4):
        total += q.get()
    print(total)

def normal(l):
    total = sum(l)
    print(total)

l = list(range(1000000))
s_t = time.time()
normal(l*4)
print('normal: ',time.time()-s_t)
s_t = time.time()
multithreading(l)
print('multithreading: ', time.time()-s_t)



1999998000000
normal:  0.07899737358093262
1999998000000
multithreading:  0.07300972938537598


## Multi-processing

In [ ]:
# import multiprocessing
# import time

# def compute_sum(n):
#     print(f"Summing up to {n}")
#     return sum(range(n))

# if __name__ == '__main__':
#     # Workload: large sums
#     tasks = [10_000_000, 10_000_000, 10_000_000, 10_000_000]

#     print("\n--- Sequential ---")
#     start = time.time()
#     results_seq = [compute_sum(n) for n in tasks]
#     end = time.time()
#     print("Results:", results_seq)
#     print(f"Sequential time: {end - start:.2f} seconds")

#     print("\n--- Multiprocessing ---")
#     start = time.time()
#     with multiprocessing.Pool(processes=4) as pool:
#         results_mp = pool.map(compute_sum, tasks)
#     end = time.time()
#     print("Results:", results_mp)
#     print(f"Multiprocessing time: {end - start:.2f} seconds")


--- Sequential ---
Summing up to 10000000
Summing up to 10000000
Summing up to 10000000
Summing up to 10000000
Results: [49999995000000, 49999995000000, 49999995000000, 49999995000000]
Sequential time: 0.66 seconds

--- Multiprocessing ---


## AsyncIO

In [ ]:
import asyncio
import time

async def task(name, delay):
    print(f"{name} started")
    await asyncio.sleep(delay)
    print(f"{name} finished after {delay}s")

async def main():
    start = time.time()

    # Run three tasks concurrently
    await asyncio.gather(
        task("Task A", 2),
        task("Task B", 1),
        task("Task C", 3)
    )

    end = time.time()
    print(f"All done in {end - start:.2f} seconds")

# Run the event loop
asyncio.run(main())

## Thread Lock

In [13]:
def job1():
    global A
    for i in range(10):
        A+=1
        print('job1',A)

In [18]:
def job2():
    global A
    for i in range(10):
        A+=10
        print('job2',A)

In [19]:
A=0
t1=threading.Thread(target=job1)
t2=threading.Thread(target=job2)
t1.start()
t2.start()
t1.join()
t2.join()

job1 1
job1 2
job1 3
job1 4
job1 5
job1 6
job1 7
job1 8
job1 9
job1 10
job2 20
job2 30
job2 40
job2 50
job2 60
job2 70
job2 80
job2 90
job2 100
job2 110


In [5]:
A

110

In [6]:
def job1():
    global A,lock
    lock.acquire()
    for i in range(10):
        A+=1
        print('job1',A)
    lock.release()

def job2():
    global A,lock
    lock.acquire()
    for i in range(10):
        A+=10
        print('job2',A)
    lock.release()

In [7]:
lock=threading.Lock()
A=0
t1=threading.Thread(target=job1)
t2=threading.Thread(target=job2)
t1.start()
t2.start()
t1.join()
t2.join()

job1 1
job1 2
job1 3
job1 4
job1 5
job1 6
job1 7
job1 8
job1 9
job1 10
job2 20
job2 30
job2 40
job2 50
job2 60
job2 70
job2 80
job2 90
job2 100
job2 110


In [37]:
counter = 0

def add():
    global counter
    for _ in range(1000000):
        counter += 1

threads = [threading.Thread(target=add) for _ in range(2)]
for t in threads:
    t.start()
for t in threads:
    t.join()

print("Final counter:", counter)  # Often less than 2,000,000

Final counter: 1846831


In [38]:
counter = 0
lock = threading.Lock()

def add():
    global counter
    for _ in range(1000000):
        with lock:
            counter += 1  # Only one thread at a time can do this

threads = [threading.Thread(target=add) for _ in range(2)]
for t in threads:
    t.start()
for t in threads:
    t.join()

print("Final counter:", counter)  # Now always 2,000,000

Final counter: 2000000
